# Tokenization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/2-tokenization.ipynb)

## Referencias
* [NLP - Natural Language Processing With Python](https://www.udemy.com/course/nlp-natural-language-processing-with-python)

Cada cuerpo de texto se compone de unos "elementos", estos elementos son unidades individuales conocidas como "tokens". Dividir el corpus en tokens es importante porque cada uno juega un papel importante y diferente a nivel semántico en el texto.

En este notebook expliraremos este concepto y como implementarlo con spacy.

In [3]:
import pkg_resources
import warnings

# Esta línea le indica a Python que ignore todas las advertencias. Si bien esto puede hacer que la salida sea más limpia,
# generalmente no se recomienda en código de producción, ya que podría pasar por alto información importante.
warnings.filterwarnings('ignore')

# Esta línea obtiene una lista de todos los paquetes instalados y extrae sus claves (nombres).
# pkg_resources está obsoleto como API, consulte https://setuptools.pypa.io/en/latest/pkg_resources.html
installed_packages = [package.key for package in pkg_resources.working_set]

# Esta línea verifica si la cadena 'google-colab' está presente en la lista de paquetes instalados.
# Si es así, significa que el código se está ejecutando en un entorno de Google Colab, y la variable IN_COLAB se establece en True.
# De lo contrario, se establece en False.
IN_COLAB = 'google-colab' in installed_packages

In [4]:
# Este comando verifica si la variable de shell IN_COLAB es igual a 'True'.
# Esta variable probablemente se estableció en una celda anterior para indicar si el entorno es Google Colab.
# El operador '&&' significa que el siguiente comando solo se ejecutará si el comando anterior (el 'test') fue exitoso.
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/cam2149/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt
# Si el entorno es Google Colab, este comando descarga el archivo requirements.txt desde la URL de GitHub especificada.
# Otro operador '&&', lo que significa que el siguiente comando se ejecutará solo si el comando 'wget' fue exitoso.
# Este comando usa pip para instalar todos los paquetes de Python listados en el archivo requirements.txt descargado.
# La bandera '-r' indica que pip debe instalar paquetes desde un archivo de requisitos.

--2025-08-07 22:08:13--  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt [following]
--2025-08-07 22:08:13--  https://raw.githubusercontent.com/Ohtar10/icesi-nlp/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     349  --.-KB/s    in 0s      

2025-08-07 22:08:13 (6.04 MB/s) - ‘requirements.txt’ saved [349/349]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import spacy
# Carga un modelo pre-entrenado en inglés. "en_core_web_sm" es un modelo pequeño que incluye tokenización,
# etiquetado de partes del habla y reconocimiento de entidades nombradas.
nlp = spacy.load("en_core_web_sm")

# Define una cadena de texto para procesar.
mystring = '"We\'re moving to L.A.!"'
# Imprime la cadena original.
print(mystring)

"We're moving to L.A.!"


## Tokenization con Spacy

Al inicio hemos dicho que los tokens son unos "elementos" que usamos para dividir el corpus. He usado la palabra elementos porque un token no está restringido únicamente a palabras, un token pueden ser n-gramas o incluso cada letra puede ser considerado un token. Todo depende de la tarea y el nivel de especificidad.

Por defecto, spacy tokeniza por palabras y algunos signos de puntuación. Observemos lo que hace Spacy con la oración que hemos definido antes.

In [3]:
# Procesa la cadena 'mystring' utilizando el objeto nlp de spaCy.
# Esto tokeniza el texto y crea un objeto Doc que contiene los tokens procesados.
doc = nlp(mystring)

# Itera a través de cada token en el objeto Doc.
for token in doc:
    # Imprime el token actual.
    print(token)

"
We
're
moving
to
L.A.
!
"


Spacy es capaz de entender textos complejos como correos electrónicos y direcciones web, además del role que cada caracter juega en la oración.

In [4]:
# Procesa una nueva cadena de texto que incluye un correo electrónico y una URL.
# Esto muestra cómo spaCy maneja estructuras de texto más complejas.
doc2 = nlp(u"We're here to help! Send snail-mail, email support@oursite.com or visit us at http://www.oursite.com!")

# Itera a través de cada token en el objeto Doc resultante.
for t in doc2:
    # Imprime el token actual.
    print(t)

We
're
here
to
help
!
Send
snail
-
mail
,
email
support@oursite.com
or
visit
us
at
http://www.oursite.com
!


Observemos que en este caso el caracter punto (.) es utilizado en el correo y la url pero no fue interpretado como un token independiente.

## Entidades
Spacy tiene el poder de inferir entidades o sustantivos.

In [5]:
# Procesa la oración para identificar entidades nombradas.
doc = nlp("Apple to build a Hong Kong factory for $6 million.")
# Itera sobre cada entidad encontrada en el documento.
for entity in doc.ents:
    # Imprime el texto de la entidad.
    print(entity)
    # Imprime la etiqueta de la entidad (el tipo de entidad).
    print(entity.label_)
    # Imprime la explicación de la etiqueta de la entidad.
    print(str(spacy.explain(entity.label_)))
    # Imprime una línea en blanco para separar las entidades.
    print('\n')

Apple
ORG
Companies, agencies, institutions, etc.


Hong Kong
GPE
Countries, cities, states


$6 million
MONEY
Monetary values, including unit




Observemos como la libreria es capaz de pre-clasificar algunas palabras en cuanto a que posiblemente hacen referencia.

##  Trozos de sustantivos
Spacy es capaz de detectar sustantivos compuestos, es decir sustantivos que están compuestos por más de una palabra.

In [6]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers.")
for chunk in doc.noun_chunks:
    # Itera sobre cada "trozo de sustantivo" (noun chunk) encontrado en el documento.
    print(chunk)

Autonomous cars
insurance liability
manufacturers


Observemos que en el ejemplo anterior "Autonomous car" es un sustantivo compuesto y spacy es capaz de identificarlo como un trozo.

Este es un caso particular en el inglés, en los diccionarios en español podrían haber otras peculiaridades.

## Displaycy

Este es un modulo para visualizar objetos spacy. Resulta muy útil para dibujar la relación semantica detectada entre los tokens.

In [7]:
from spacy import displacy

doc = nlp("Apple is going to build a U.K. factory for $6 million.")
# dep for syntactic dependency
# Este código utiliza displacy para visualizar las dependencias sintácticas de una oración.
# 'doc' es el objeto Doc de spaCy que contiene la oración procesada.
# style='dep' especifica que se visualicen las dependencias.
# jupyter=True permite renderizar la visualización directamente en un entorno Jupyter o Colab.
# options={'distance': 110} ajusta la distancia entre los tokens en la visualización para mejorar la legibilidad.
displacy.render(doc, style='dep', jupyter=True, options={'distance': 110})

In [8]:
doc = nlp("Over the last quarter Apple sold nearly 20 thousand iPds for  profit of $6 million.")
# dep for syntactic dependency
# Este código utiliza displacy para visualizar las entidades nombradas (NER) de una oración.
# 'doc' es el objeto Doc de spaCy que contiene la oración procesada con las entidades identificadas.
# style='ent' especifica que se visualicen las entidades nombradas.
# jupyter=True permite renderizar la visualización directamente en un entorno Jupyter o Colab.
displacy.render(doc, style='ent', jupyter=True)

In [10]:
%pip install spacytextblob

In [20]:
import pandas as pd
import csv
import pandas as pd
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

file_path = "/content/quejas.csv"
data = []

# Función para analizar el estado de ánimo
def analyze_sentiment(text):
    doc = nlp(text)
    polarity = doc._.blob.polarity  # Puntuación de polaridad (-1 a 1)
    if polarity > 0:
        sentiment_label = "Positivo"
    elif polarity < 0:
        sentiment_label = "Negativo"
    else:
        sentiment_label = "Neutral"
    return polarity, sentiment_label

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        # Use the csv module to handle potential complexities
        reader = csv.reader(f)
        for row in reader:
            # Assuming the structure is always id, name, description
            if len(row) >= 3:
                id_queja = row[0]
                nombreCliente = row[1]
                # Join the remaining elements to form the description
                descripcion = ','.join(row[2:])
                data.append([id_queja, nombreCliente, descripcion])
            elif len(row) > 0:
                 # Handle lines with fewer than expected fields if necessary
                 print(f"Skipping incomplete line: {row}")


    # Create a DataFrame from the parsed data
    df = pd.DataFrame(data, columns=['id_queja', 'nombreCliente', 'Descripcion'])

    # Display the first few rows and column names to verify
    print(df.head())
    print(df.columns)

    # Aplicar el análisis de sentimientos a la columna 'Descripcion'
    df['Polarity'], df['Sentiment'] = zip(*df['Descripcion'].apply(analyze_sentiment))

    # Guardar los resultados en un nuevo CSV
    df.to_csv("/content/quejas_con_sentimientos.csv", index=False)

    # Display the first few rows and column names to verify
    print(df.head())
    print(df.columns)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

   id_queja  nombreCliente                                        Descripcion
0  id_queja  nombreCliente                                        Descripcion
1         1     Juan Pérez  The product arrived broken and the customer se...
2         2    María Gómez  I loved the product, it exceeded my expectations.
3         3   Carlos López  The delivery was on time, but the packaging wa...
Index(['id_queja', 'nombreCliente', 'Descripcion'], dtype='object')
   id_queja  nombreCliente                                        Descripcion  \
0  id_queja  nombreCliente                                        Descripcion   
1         1     Juan Pérez  The product arrived broken and the customer se...   
2         2    María Gómez  I loved the product, it exceeded my expectations.   
3         3   Carlos López  The delivery was on time, but the packaging wa...   

   Polarity Sentiment  
0       0.0   Neutral  
1      -0.7  Negativo  
2       0.7  Positivo  
3       0.0   Neutral  
Index(['id_queja